In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [2]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [3]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 64
    # bs = 10
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [4]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [5]:
from model.dataset import PlanTreeDataset

## TEST - Loading 10 tensors

In [6]:
import os
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from model.dataset import PlanTreeDataset  # Assuming PlanTreeDataset is defined elsewhere
import json

# Path to the tensors folder
tensors_dir = "./job_queries/tensors"

# Validate if the tensors directory exists
if not os.path.exists(tensors_dir):
    raise FileNotFoundError(f"Tensors directory '{tensors_dir}' not found.")

# Get all tensor file paths from the directory
tensor_files = sorted(os.listdir(tensors_dir))

# Validate if tensor files exist in the directory
if not tensor_files:
    raise FileNotFoundError(f"No tensor files found in '{tensors_dir}'.")

# Initialize lists to store tensor components
x_list = []
rel_pos_list = []
attn_bias_list = []
heights_list = []
cost_labels_list = []
raw_costs_list = []

# Load tensors dynamically
for tensor_file in tensor_files:
    tensor_path = os.path.join(tensors_dir, tensor_file)
    try:
        loaded_tensors = torch.load(tensor_path)
        # Append components to respective lists
        x_list.append(loaded_tensors["x"])
        rel_pos_list.append(loaded_tensors["rel_pos"])
        attn_bias_list.append(loaded_tensors["attn_bias"])
        heights_list.append(loaded_tensors["heights"])
        cost_labels_list.append(loaded_tensors["cost_labels"])
        raw_costs_list.append(loaded_tensors["raw_costs"])
    except Exception as e:
        print(f"Error loading tensor file '{tensor_file}': {e}")

# Generate indices for splitting
num_examples = len(x_list)
if num_examples == 0:
    raise ValueError("No valid tensors loaded for dataset creation.")
    
all_indices = np.arange(num_examples)

# Perform train-validation split with fixed seed for reproducibility
train_indices, val_indices = train_test_split(all_indices, test_size=0.2, random_state=0)

# Create training and validation datasets
train_dataset = PlanTreeDataset(
    len(train_indices),
    [x_list[i] for i in train_indices],
    [attn_bias_list[i] for i in train_indices],
    [rel_pos_list[i] for i in train_indices],
    [heights_list[i] for i in train_indices],
    [cost_labels_list[i] for i in train_indices],
    [raw_costs_list[i] for i in train_indices]
)

val_dataset = PlanTreeDataset(
    len(val_indices),
    [x_list[i] for i in val_indices],
    [attn_bias_list[i] for i in val_indices],
    [rel_pos_list[i] for i in val_indices],
    [heights_list[i] for i in val_indices],
    [cost_labels_list[i] for i in val_indices],
    [raw_costs_list[i] for i in val_indices]
)

# Save validation indices and file names
val_file_names = [tensor_files[i] for i in val_indices]
val_data = {
    "val_indices": val_indices.tolist(),
    "file_names": val_file_names,
}

val_data_file = "./val_data.json"
with open(val_data_file, "w") as f:
    json.dump(val_data, f)

# Save training indices and file names
train_file_names = [tensor_files[i] for i in train_indices]
train_data = {
    "train_indices": train_indices.tolist(),
    "file_names": train_file_names,
}

train_data_file = "./train_data.json"
with open(train_data_file, "w") as f:
    json.dump(train_data, f)

# Print information
print("Training Dataset length:", len(train_dataset))
print("Validation Dataset length:", len(val_dataset))
print(f"Validation data saved to {val_data_file}")
print(f"Training data saved to {train_data_file}")

Training Dataset length: 1865
Validation Dataset length: 467
Validation data saved to ./val_data.json
Training data saved to ./train_data.json


In [7]:
import pandas as pd
import json

# File paths
val_data_file = "./val_data.json"
train_data_file = "./train_data.json"

# Load validation data from JSON file
with open(val_data_file, "r") as f:
    loaded_val_data = json.load(f)

loaded_val_indices = loaded_val_data["val_indices"]
loaded_val_file_names = loaded_val_data["file_names"]

# Create a DataFrame for validation data
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names
})

# Display the Validation DataFrame
print("Validation DataFrame:")
print(val_data_df.head())

# Save validation data to CSV
val_data_df.to_csv("./val_data.csv", index=False)
print("Validation data saved to ./val_data.csv")

# Load training data from JSON file
with open(train_data_file, "r") as f:
    loaded_train_data = json.load(f)

loaded_train_indices = loaded_train_data["train_indices"]
loaded_train_file_names = loaded_train_data["file_names"]

# Create a DataFrame for training data
train_data_df = pd.DataFrame({
    "train_index": loaded_train_indices,
    "filename": loaded_train_file_names
})

# Display the Training DataFrame
print("Training DataFrame:")
print(train_data_df.head())

# Save training data to CSV
train_data_df.to_csv("./train_data.csv", index=False)
print("Training data saved to ./train_data.csv")


Validation DataFrame:
   val_index                                  filename
0       1056  query_1952_2024-12-03-21.34.02.429279.pt
1       2323   query_993_2024-12-03-21.10.08.618148.pt
2        655  query_1591_2024-12-03-21.23.17.861948.pt
3       2101   query_793_2024-12-03-21.05.49.462784.pt
4       1401  query_2262_2024-12-03-21.44.00.539251.pt
Validation data saved to ./val_data.csv
Training DataFrame:
   train_index                                  filename
0          812  query_1732_2024-12-03-21.27.45.652122.pt
1          233  query_1210_2024-12-03-21.14.50.599373.pt
2         1380  query_2243_2024-12-03-21.43.30.935782.pt
3         1752   query_479_2024-12-03-20.59.07.992844.pt
4         1334  query_2201_2024-12-03-21.42.07.400700.pt
Training data saved to ./train_data.csv


In [8]:
len(train_dataset)

1865

In [9]:
len(train_dataset[:10])

10

In [10]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, train_dataset, val_dataset, crit, cost_norm, args)


running epoch: 0


pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([64, 1])


...

pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([64, 1])


pred_output shape: torch.Size([19, 1])
ps: [35268.89   13455.445  35224.688  12253.904  67124.     13057.194
 67126.11   64274.676  35120.266  12242.937  67125.98   12240.671
 64274.12   64281.293  67125.98   64277.37   35124.42   67125.98
 64279.516  35361.035  64277.492  35242.492  14827.863  64269.77
 67125.22   67125.28   12244.523  67125.15   13451.096  67123.74
 64271.855  64278.1    35179.465  64277.242  67125.28   26790.066
 67125.86   35195.105  12242.434  64278.715  64277.125  67125.28
 67125.15   64275.836  64266.89   35163.535  67123.94   35120.6
 64270.266  12706.599  12242.725  12238.371  13052.152  64273.33
 35124.855  35229.96   64278.598  67124.9    67125.73   67123.55
 67124.96   12242.341  35124.29   12242.469  64275.594  67125.92
 64273.754  67124.     12226.81   12235.861  67125.86   67122.84
 35265.63   12241.43   35267.547  64275.652  12242.294  67123.74
 13045.471  67125.92   67126.11   13052.912  12246.007  13059.747
 12231.078  12708.962  64278.348  67125.86  

ls: [tensor([35153.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([38815.], dtype=torch.float64), tensor([14175.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([58513.], dtype=torch.float64), tensor([39421.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([14187.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([66822.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([35343.], dtype=torch.float64), tensor([66823.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([19798.], dtype=torch.float64), tensor([58512.], dtype=torch.float64), tensor([38590.], dtype=torch.float64), tensor([9951.], dtype=torch.float64), tensor([58513.], dtype=torch.float64), tensor([66821.], dtype=torch.float64), tensor([66821.], dtype=t

In [11]:
best_model_path

'-3799469494494950040.pt'

In [12]:
len(loaded_val_indices[:10])

10

In [13]:
len(val_embeddings)

467

In [14]:
import pandas as pd
import numpy as np

# Assume these are loaded or computed
# val_embeddings: List or numpy array of embeddings (e.g., shape [num_examples, embedding_dim])
# loaded_val_indices: List of validation indices
# loaded_file_names: List of validation file names

# Convert val_embeddings to a numpy array if it's not already
val_embeddings = np.array(val_embeddings)

# Ensure the size of all inputs matches
if len(loaded_val_indices) != len(loaded_val_file_names) or len(loaded_val_indices) != len(val_embeddings):
    raise ValueError("The sizes of validation indices, filenames, and embeddings must match.")

# Create a DataFrame
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names,
    "embedding": list(val_embeddings)  # Store embeddings as a list of numpy arrays
})

# Save the DataFrame to a CSV file
csv_file_path = "validation_embeddings.csv"
val_data_df.to_csv(csv_file_path, index=False)

# Optionally save embeddings in a separate binary file for efficient storage
embeddings_file_path = "validation_embeddings.npy"
np.save(embeddings_file_path, val_embeddings)

print(f"Validation DataFrame created and saved to {csv_file_path}")
print(f"Embeddings saved to {embeddings_file_path}")

# Print the first few rows for verification
print(val_data_df.head())


Validation DataFrame created and saved to validation_embeddings.csv
Embeddings saved to validation_embeddings.npy
   val_index                                  filename  \
0       1056  query_1952_2024-12-03-21.34.02.429279.pt   
1       2323   query_993_2024-12-03-21.10.08.618148.pt   
2        655  query_1591_2024-12-03-21.23.17.861948.pt   
3       2101   query_793_2024-12-03-21.05.49.462784.pt   
4       1401  query_2262_2024-12-03-21.44.00.539251.pt   

                                           embedding  
0  [-114.55693, -101.882355, -14.314699, -1.86743...  
1  [-104.10589, -100.892784, -15.305394, 19.10723...  
2  [-114.54547, -101.893394, -14.308007, -1.88108...  
3  [-89.35007, -103.53369, -24.114931, 38.514816,...  
4  [-99.47161, -60.7916, -35.936123, -1.4182578, ...  


In [15]:
val_data_df.head()

,val_index,filename,embedding
0,1056,query_1952_2024-12-03-21.34.02.429279.pt,"[-114.55693, -101.882355, -14.314699, -1.86743..."
1,2323,query_993_2024-12-03-21.10.08.618148.pt,"[-104.10589, -100.892784, -15.305394, 19.10723..."
2,655,query_1591_2024-12-03-21.23.17.861948.pt,"[-114.54547, -101.893394, -14.308007, -1.88108..."
3,2101,query_793_2024-12-03-21.05.49.462784.pt,"[-89.35007, -103.53369, -24.114931, 38.514816,..."
4,1401,query_2262_2024-12-03-21.44.00.539251.pt,"[-99.47161, -60.7916, -35.936123, -1.4182578, ..."


In [16]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 1865
First training embedding shape: (1417,)
Number of validation embeddings (best epoch): 467
First validation embedding shape: (1417,)


In [17]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [-114.55693  -101.882355  -14.314699 ...   51.675053   26.12046
   56.93147 ]


In [18]:
type(val_embeddings[0])

numpy.ndarray

In [19]:
print(f"First validation embedding shape: {val_embeddings[1]}")

First validation embedding shape: [-104.10589  -100.892784  -15.305394 ...   48.47303    35.593414
   74.74174 ]


In [20]:
len(val_embeddings[0])

1417

In [21]:
# import numpy as np

# # Disable truncation
# np.set_printoptions(threshold=np.inf, linewidth=1000)

# for i, emb in enumerate(val_embeddings):
#     print(f"Full Embedding {i}:\n{emb}")


In [22]:
import json
import numpy as np

# Ensure embeddings are numpy arrays
train_embeddings = np.array(train_embeddings)
val_embeddings = np.array(val_embeddings)

# Paths for saving the JSON files
train_embeddings_file = "./train_embeddings.json"
val_embeddings_file = "./val_embeddings.json"

# Create dictionaries with file names as keys and embeddings as values
train_embedding_dict = {train_file_name: embedding.tolist() for train_file_name, embedding in zip(loaded_train_file_names, train_embeddings)}
val_embedding_dict = {val_file_name: embedding.tolist() for val_file_name, embedding in zip(loaded_val_file_names, val_embeddings)}

# Save training embeddings to JSON
with open(train_embeddings_file, "w") as f:
    json.dump(train_embedding_dict, f)

# Save validation embeddings to JSON
with open(val_embeddings_file, "w") as f:
    json.dump(val_embedding_dict, f)

print(f"Train embeddings saved to {train_embeddings_file}")
print(f"Validation embeddings saved to {val_embeddings_file}")

Train embeddings saved to ./train_embeddings.json
Validation embeddings saved to ./val_embeddings.json


In [23]:
import pandas as pd
import json

# Path to the JSON file
val_embeddings_file = "./val_embeddings.json"  # Replace with train_embeddings.json for training data

# Load the JSON file
with open(val_embeddings_file, "r") as f:
    embeddings_data = json.load(f)

# Convert JSON data to a DataFrame
val_embeddings_df = pd.DataFrame({
    "file_name": list(embeddings_data.keys()),
    "embedding": list(embeddings_data.values())
})

# Display the DataFrame
print(val_embeddings_df.head())

# Optionally save it to a CSV
val_embeddings_df.to_csv("val_embeddings.csv", index=False)
print("Validation embeddings saved to val_embeddings.csv")


                                  file_name  \
0  query_1952_2024-12-03-21.34.02.429279.pt   
1   query_993_2024-12-03-21.10.08.618148.pt   
2  query_1591_2024-12-03-21.23.17.861948.pt   
3   query_793_2024-12-03-21.05.49.462784.pt   
4  query_2262_2024-12-03-21.44.00.539251.pt   

                                           embedding  
0  [-114.55693054199219, -101.88235473632812, -14...  
1  [-104.10588836669922, -100.89278411865234, -15...  
2  [-114.54547119140625, -101.89339447021484, -14...  
3  [-89.35006713867188, -103.53369140625, -24.114...  
4  [-99.47161102294922, -60.79159927368164, -35.9...  


Validation embeddings saved to val_embeddings.csv
